In [1]:
# required Python imports
import pandas as pd
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.3.19 imported


## Generate database

In [2]:
f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179))

array([[  0,   0,   0],
       [  0,   0,   7],
       [  0,   0,  14],
       ...,
       [ 84,  84, 161],
       [ 84,  84, 168],
       [ 84,  84, 175]])

In [3]:
# generate Euler angles each 7 degrees and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))
database.euler.shape

(4394, 3)

In [4]:
# Transmission values for lambda 1987.29
Ta = 0.009882
Tb = 0.995815
Tc = 0.596951

# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(Ta, Tb, Tc), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

# Generate 16 measures at different azimuth angles
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [5]:
# check
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(4394, 3)
(4394, 16)
(4394, 16)


In [6]:
# check
print('Euler angles: ', database.euler[-1])
print('T values: ', np.around(database.T_values[-1], 2))
print('Azimuths: ', np.around(database.azimuths[-1], 1))

Euler angles:  [ 84  84 175]
T values:  [0.98 0.89 0.74 0.62 0.6  0.68 0.83 0.96 0.98 0.89 0.74 0.62 0.6  0.68
 0.83 0.96]
Azimuths:  [  0.2  22.5  45.2  67.4  90.  112.3 135.  157.5 180.  202.5 225.2 247.5
 270.  292.3 315.  337.5]


## Estimate

In [9]:
# initialize variables 
default20_algorithm = np.empty(database.euler.shape[0])
diffEvol_algorithm = np.empty(database.euler.shape[0])
annealing_algorithm = np.empty(database.euler.shape[0])

for index, orientation in enumerate(database.euler):
    # print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    estimate_1 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc), num_guesses=10)
    default20_algorithm[index] = f.calc_disorientation(orientation, estimate_1.x)

    estimate_2 = f.find_orientation_diffevol(measurements=measures, params=(Ta, Tb, Tc))
    diffEvol_algorithm[index] = f.calc_disorientation(orientation, estimate_2.x)

    estimate_3 = f.find_orientation_annealing(measurements=measures, params=(Ta, Tb, Tc))
    annealing_algorithm[index] = f.calc_disorientation(orientation, estimate_3.x)


> in my PC each loop (i.e. orientation) takes ~1.7 s so that n=4394 means ~124m (2 h)

## Generate dataframe and save data

In [11]:
dataset = pd.DataFrame(
    {
        'Euler1': database.euler[:, 0],
        'Euler2': database.euler[:, 1],
        'Euler3': database.euler[:, 2],
        'default20': default20_algorithm,
        'diffEvol': diffEvol_algorithm,
        'Anne': annealing_algorithm,
    }
)

dataset

,Euler1,Euler2,Euler3,default10,default20,default30
0,0,0,0,0.162,0.190,0.223
1,0,0,7,0.215,0.210,0.214
2,0,0,14,0.197,0.209,0.215
3,0,0,21,0.212,0.205,0.196
4,0,0,28,0.220,0.213,0.216
...,...,...,...,...,...,...
4389,84,84,147,0.004,0.004,0.003
4390,84,84,154,0.004,0.004,0.004
4391,84,84,161,0.003,0.003,0.003
4392,84,84,168,0.004,0.004,0.004


In [13]:
from datetime import date    
today = date.today().isoformat()

In [14]:
dataset.to_csv(today + '_step7_lamb1987_diffAlgo.csv', index=False)

In [12]:
# quick statistical overview
dataset[['default20', 'diffEvol', 'Anne']].describe()

,default10,default20,default30
count,4394.000000,4394.000000,4394.000000
mean,0.230177,0.067985,0.045864
std,1.534805,0.586355,0.135742
min,0.000000,0.000000,0.000000
25%,0.001000,0.001000,0.001000
50%,0.003000,0.003000,0.003000
75%,0.008000,0.007000,0.007000
max,31.485000,27.828000,3.300000
